# Hermitage Odoo migration notebook

This notebook app will run the ETL process to migrate Heremitage's Odoo 11 to a new Odoo 15 system.
It will basically extract the data from v11 into csv files, make the necessary transformations to make it importable in Odoo 15 and then load the data it into Hermitage's new Odoo 15 instance.

## Initial Configurations

**Export Configurations File**
In this workspace you can find the file [export_connection.conf](export_connection.conf), it is used to set all the necessary settings and credentials to connect to Hermitage's Odoo 11 instance

**Import Configurations File**
In this workspace you can find the file [import_connection.conf](import_connection.conf), it is used to set all the necessary settings and credentials to connect to Hermitage's Odoo 15 instance

**Models Migration Configurations File**
In this workspace you can find the file [models_migration_config.conf](models_migration_config.conf). In it its possible to change certain aspects of how the models are extracted and loaded into the new system.
Most common possible configurations are:
- Fields to migrate
- Domain/Filter used to extract
- Fields to ignore when loading the data

**States Remapping File**
In this workspace you can find the file [input_csv_files/States Remapping - Input.csv](input_csv_files/States%20Remapping%20-%20Input.csv.conf), it is used to change the states and countries from certain partners.
This file is extracted from a Google Spreadsheet where some states and country data was manually corrected and then mapped with formulas to Odoo's 15 states and country base data.
Link to the Spreadsheet to check which states were affected: [States Remapping](https://docs.google.com/spreadsheets/d/1hL9S4APfv9cJYEihGn9Rtimt0xQp7mq1E0ZRpWafa8I/edit?usp=share_link)

### Imports
- pandas: to make transformations on the data
- odoo_csv_tools: Odoo SDK python package specialized for ETL processes
- Models' migration config
- Import function

### Constants
No need to parameterize these values for now
Included file paths for some extra csv files that don't follow the standard naming convention of model_name.csv, e.g: res.partner.csv

**Export function wrapper**

In [1]:
import pandas

from odoo_csv_tools import export_threaded

from models_migration_config import models_migration_config
from import_functions import import_data, import_ignored_fields

EXPORT_CONNECTION_CONFIG_DIR = 'export_connection.conf'
EXPORT_DEFAULT_BATCH_SIZE = 3000
EXPORT_DEFAULT_REQ_CONTEXT = {}
DEFAULT_WORKERS = 2

GENERATED_CSV_FILES_PATH = 'generated_csv_files/'
INPUT_CSV_FILES_PATH = 'input_csv_files/'
STATES_REMAPPING_FILE_NAME = 'States Remapping - Input.csv'
PARTNERS_WITHOUT_NAME_FILE_NAME = 'res.partner(no name).csv'
RES_USERS_GROUPS_FILE_NAME = 'res.users(groups).csv'
CRM_TEAM_MEMBERS_FILE_NAME = 'crm.team(members).csv'

national_accounts_branch_external_id = "multi_branches.main_branch"
design_center_branch_external_id = "__export__.res_branch_4_e6b8f9eb"

old_national_accounts_branch_external_id = "__export__.res_branch_2_45e420c1"
old_kitchen_branch_external_id = "__export__.res_branch_6_734443bb"
old_gallery_531_branch_external_id = "__export__.res_branch_1_d965e632"

def export_data(model_name = None, config = None, domain = None, fields = None, output_file = None, workers = None, batch_size = None, context = None, separator = None):
    if model_name in ['ir.model.data', 'ir.model.fields', 'res.groups']:
        model_migration_config = {}
    else:
        model_migration_config = models_migration_config[model_name]
    if not config:
        config = EXPORT_CONNECTION_CONFIG_DIR
    if not domain:
        domain = model_migration_config.get('domain', [])
    if not fields:
        fields = model_migration_config.get('fields', [])
    if output_file:
        output_file = f'{GENERATED_CSV_FILES_PATH}{output_file}'
    else:
        output_file = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
    if not workers:
        workers = DEFAULT_WORKERS
    if not batch_size:
        batch_size = model_migration_config.get('batch_size', EXPORT_DEFAULT_BATCH_SIZE)
    if not context:
        context = model_migration_config.get('context', EXPORT_DEFAULT_REQ_CONTEXT)
    if not separator:
        separator = model_migration_config.get('separator', ',')

    export_threaded.export_data(
        config,
        model_name,
        domain,
        fields,
        output=output_file,
        max_connection=workers,
        batch_size=batch_size,
        context=context,
        separator=separator,
    )

def replace_branches_ids(df):
    df['branch_id/id'] = df['branch_id/id'].str.replace(old_national_accounts_branch_external_id, national_accounts_branch_external_id)
    df['branch_id/id'] = df['branch_id/id'].str.replace(old_kitchen_branch_external_id, design_center_branch_external_id)
    df['branch_id/id'] = df['branch_id/id'].str.replace(old_gallery_531_branch_external_id, design_center_branch_external_id)


**Extract res.partner.category**

In [ ]:
export_data('res.partner.category')

**Load account.payment.term**

In [ ]:
model_name = 'account.payment.term'

import_data(model_name, absolute_file_path=f'{INPUT_CSV_FILES_PATH}{model_name}.csv',
            ignore_fields=['discount_days', 'payment_term_for'])
# Ignoring the fileds that are not implemented.

**Load res.partner.category**

In [ ]:
import_data('res.partner.category')

**Extract and transform res.partner**

In [ ]:
model_name = 'res.partner'
model_migration_config = models_migration_config[model_name]

export_data(model_name)

partners_main_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
partners_dataframe = pandas.read_csv(partners_main_file_path)

# States remapping
states_remapping_file_path = f'{INPUT_CSV_FILES_PATH}{STATES_REMAPPING_FILE_NAME}'
states_remapping_dataframe = pandas.read_csv(states_remapping_file_path)

def transformations(df: pandas.DataFrame) -> None:
    df.rename(columns={
        'categ_id/id': 'category_id/id',
        'supplier': 'supplier_rank',
        'customer': 'customer_rank',
        'hesco_account_number': 'customer_account_number',
        'hesco_account_open_date': 'customer_account_open_date'
    }, inplace=True)

    df['category_id/id'] = df['category_id/id'].str.replace('False', '')
    df['type'] = df['type'].str.replace('False', '')
    df['type'] = df['type'].str.replace('Shipping address', 'Delivery address')
    # df['property_supplier_payment_term_id/name'] = df['property_supplier_payment_term_id/name'].str.replace('False', '')

    df['supplier_rank'] = df['supplier_rank'].astype('str')
    df['customer_rank'] = df['customer_rank'].astype('str')
    df['supplier_rank'] = df['supplier_rank'].str.replace('False', '0')
    df['supplier_rank'] = df['supplier_rank'].str.replace('True', '0')
    df['customer_rank'] = df['customer_rank'].str.replace('False', '0')
    df['customer_rank'] = df['customer_rank'].str.replace('True', '1')
    df['supplier_rank'] = df['supplier_rank'].astype('int')
    df['customer_rank'] = df['customer_rank'].astype('int')

    for row in states_remapping_dataframe.itertuples():
        indexes = df[df['state_id/id'] == row.flash_hlg_state_id].index.tolist()
        for i in indexes:
            state_id = row.id if type(row.id) == str else ''
            df.loc[i:i, 'state_id/id': 'country_id/id'] = state_id, row.country_id

transformations(partners_dataframe)
partners_dataframe.to_csv(partners_main_file_path, index=False)

#Export partner without names
fields_to_export = [f for f in model_migration_config['fields'] if f != 'name']
export_data(model_name=model_name,
            domain=['|', ['name', '=', False], ['name', '=', ''], ['customer', '=', True], ['create_date', '>', '2022-11-01 00:00:00']],
            fields=fields_to_export,
            output_file=PARTNERS_WITHOUT_NAME_FILE_NAME
            )

partners_without_name_file_path = f'{GENERATED_CSV_FILES_PATH}{PARTNERS_WITHOUT_NAME_FILE_NAME}'
partners_dataframe = pandas.read_csv(partners_without_name_file_path)

partners_dataframe.insert(len(fields_to_export), 'name','[N/A]')
transformations(partners_dataframe)

partners_dataframe.to_csv(partners_without_name_file_path, index=False)

**Load res.partner**

In [4]:
model_name = 'res.partner'
#import partners without name
#import_data(model_name=model_name, file_csv=PARTNERS_WITHOUT_NAME_FILE_NAME)
#import the rest of the partners
#import_data(model_name=model_name, group_by='parent_id/id', workers=1)

ignore_fields = ['id', 'name'] + models_migration_config['res.partner']['ignore_fields']
import_ignored_fields(model_name, group_by='parent_id/id', workers=1)
#import_ignored_fields(model_name, file_csv=PARTNERS_WITHOUT_NAME_FILE_NAME, group_by='parent_id/id', ignore_fields = ignore_fields, workers=1)

open generated_csv_files/res.partner.csv


open generated_csv_files/res.partner.csv


Skipping until line 0 excluded
time for batch [0] - [False] - 1000 of 36948 : 65.54566478729248
time for batch [0] - [False] - 2000 of 36948 : 63.86566138267517
time for batch [0] - [False] - 3000 of 36948 : 62.95938563346863


batch [0] - [False], 3
{'rows': {'from': 893, 'to': 893}, 'type': 'error', 'record': 893, 'field': 'user_id', 'message': "No matching record found for external id '__export__.res_users_24' in field 'Salesperson'", 'moreinfo': {'name': 'Possible Values', 'type': 'ir.actions.act_window', 'target': 'new', 'view_mode': 'tree,form', 'views': [[False, 'list'], [False, 'form']], 'context': {'create': False}, 'help': 'See all possible values', 'res_model': 'ir.model.data', 'domain': [['model', '=', 'res.users']]}, 'field_name': 'Salesperson'}
['__export__.res_users_24_res_partner', '1', '0', '__export__.res_partner_category_91', 'False', '__export__.res_users_24', '', '']


time for batch [0] - [False] - 4000 of 36948 : 62.7231879234314
time for batch [0] - [False] - 5000 of 36948 : 66.07800006866455


...

time for batch [0] - [False] - 34000 of 36948 : 62.955097913742065
time for batch [0] - [False] - 35000 of 36948 : 63.206645488739014
time for batch [0] - [False] - 36000 of 36948 : 64.14525747299194
time for batch [0] - [False] - 37000 of 36948 : 61.27396249771118
time for batch [1] - [__export__.res_partner_2301_16e91ad6] - 1000 of 1000 : 105.6785204410553
time for batch [2] - [__export__.res_partner_29428_b9112447] - 1000 of 1001 : 104.12946271896362
time for batch [2] - [__export__.res_partner_29428_b9112447] - 2000 of 1001 : 1.028820514678955


batch [3] - [__export__.res_partner_5086_890a7298], 0
{'rows': {'from': 413, 'to': 413}, 'type': 'error', 'record': 413, 'field': 'parent_id', 'message': "No matching record found for external id '__export__.res_partner_31436_c211ba77' in field 'Related Company'", 'moreinfo': {'name': 'Possible Values', 'type': 'ir.actions.act_window', 'target': 'new', 'view_mode': 'tree,form', 'views': [[False, 'list'], [False, 'form']], 'context': {'create': False}, 'help': 'See all possible values', 'res_model': 'ir.model.data', 'domain': [['model', '=', 'res.partner']]}, 'field_name': 'Related Company'}
['__export__.res_partner_69225_32f71491', '1', '0', '__export__.res_partner_category_145_f4b29097', '__export__.res_partner_31436_c211ba77', 'False', '7461974678', '']
batch [3] - [__export__.res_partner_5086_890a7298], 0
{'rows': {'from': 440, 'to': 440}, 'type': 'error', 'record': 440, 'field': 'parent_id', 'message': "No matching record found for external id '__export__.res_partner_31637_7b0ad1d9

time for batch [3] - [__export__.res_partner_5086_890a7298] - 1000 of 1000 : 97.29499745368958


batch [4] - [__export__.res_partner_68897_5d57858f], 0
{'rows': {'from': 12, 'to': 12}, 'type': 'error', 'record': 12, 'field': 'parent_id', 'message': "No matching record found for external id '__export__.res_partner_50945_18e9824b' in field 'Related Company'", 'moreinfo': {'name': 'Possible Values', 'type': 'ir.actions.act_window', 'target': 'new', 'view_mode': 'tree,form', 'views': [[False, 'list'], [False, 'form']], 'context': {'create': False}, 'help': 'See all possible values', 'res_model': 'ir.model.data', 'domain': [['model', '=', 'res.partner']]}, 'field_name': 'Related Company'}
['__export__.res_partner_69205_d82d2766', '1', '0', '__export__.res_partner_category_145_f4b29097', '__export__.res_partner_50945_18e9824b', 'False', '2438268590', '']


time for batch [4] - [__export__.res_partner_68897_5d57858f] - 1000 of 1003 : 107.60282683372498
time for batch [4] - [__export__.res_partner_68897_5d57858f] - 2000 of 1003 : 0.9667394161224365
time for batch [5] - [base.main_partner] - 1000 of 790 : 89.16442704200745
41742 res.partner imported, total time 2856.329431295395 second(s)


**Extract and transform res.users**

In [ ]:
model_name = 'res.users'
export_data(model_name)

res_users_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
users_dataframe = pandas.read_csv(res_users_file_path)
users_dataframe.rename(columns={'image ': 'image_1920'}, inplace=True)
users_dataframe.to_csv(res_users_file_path, index=False)

# Export users security groups
fields_to_export = ['id', 'groups_id', 'groups_id/id']
export_data(model_name=model_name,
            fields=fields_to_export,
            output_file=RES_USERS_GROUPS_FILE_NAME,
            )
res_users_groups_file_path = f'{GENERATED_CSV_FILES_PATH}{RES_USERS_GROUPS_FILE_NAME}'
users_groups_dataframe = pandas.read_csv(res_users_groups_file_path)
users_groups_dataframe.fillna(method='ffill', inplace=True)

# Export v15 groups
export_data(config='import_connection.conf',
            model_name='res.groups',
            fields=['id'],
            output_file='res.groups.csv',
            )
res_groups_file_path = f'{GENERATED_CSV_FILES_PATH}res.groups.csv'
groups_dataframe = pandas.read_csv(res_groups_file_path)
groups_dataframe.rename(columns={'id': 'groups_id/id'}, inplace=True)
users_groups_dataframe = users_groups_dataframe.merge(groups_dataframe, on='groups_id/id', how='inner')

users_groups_dataframe.to_csv(res_users_groups_file_path, index=False)

**Load res.users** (and load remaining res.partner fields)

In [ ]:
model_name = 'res.users'
import_data(model_name=model_name)
import_data(model_name=model_name, ignore_fields=['groups_id'], file_csv=RES_USERS_GROUPS_FILE_NAME, group_by='id', workers=1, context={'update_many2many': True})

#Import remaining res.partner fields
ignore_fields = ['id', 'name'] + models_migration_config['res.partner']['ignore_fields']
import_ignored_fields('res.partner', ignore_fields = ignore_fields, workers=1)

**Extract crm.lead.tag**

In [ ]:
export_data('crm.lead.tag')

**Load crm.lead.tag**

In [ ]:
import_data('crm.lead.tag')

**Extract crm.stage**

In [ ]:
export_data('crm.stage')

stages_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}crm.stage.csv')

#replace company_division_id/id with branch_id/id
stages_dataframe['company_division_id/id'] = stages_dataframe['company_division_id/id'].str.replace('__export__.company_subdivision_8_1c96d77e', national_accounts_branch_external_id)
stages_dataframe['company_division_id/id'] = stages_dataframe['company_division_id/id'].str.replace('__export__.company_subdivision_2_f8b15617', design_center_branch_external_id)
stages_dataframe['company_division_id/id'] = stages_dataframe['company_division_id/id'].str.replace('__export__.company_subdivision_5_ad79c78f', design_center_branch_external_id)
stages_dataframe['company_division_id/id'] = stages_dataframe['company_division_id/id'].str.replace('__export__.company_subdivision_6_9087ebea', design_center_branch_external_id)
stages_dataframe['company_division_id/id'] = stages_dataframe['company_division_id/id'].str.replace('__export__.company_subdivision_7_b737099f', design_center_branch_external_id)
stages_dataframe['company_division_id/id'] = stages_dataframe['company_division_id/id'].str.replace('__export__.company_subdivision_27_08149b9d', design_center_branch_external_id)
stages_dataframe['company_division_id/id'] = stages_dataframe['company_division_id/id'].str.replace('__export__.company_subdivision_29_7c7a3815', design_center_branch_external_id)

stages_dataframe.rename(columns={'company_division_id/id': 'branch_id/id'}, inplace=True)

stages_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}crm.stage.csv', index=False)


**Load crm.stage**

In [ ]:
import_data('crm.stage')

**Extract and transform crm.team**

In [ ]:
model_name = 'crm.team'

export_data(model_name)
crm_team_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}{model_name}.csv')

replace_branches_ids(crm_team_dataframe)

crm_team_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}{model_name}.csv', index=False)

#Export team members
fields_to_export = ['id', 'member_ids', 'member_ids/id']
export_data(model_name=model_name,
            fields=fields_to_export,
            output_file=CRM_TEAM_MEMBERS_FILE_NAME
            )
crm_team_file_path = f'{GENERATED_CSV_FILES_PATH}{CRM_TEAM_MEMBERS_FILE_NAME}'
crm_team_dataframe = pandas.read_csv(crm_team_file_path)
crm_team_dataframe.fillna(method='ffill', inplace=True)
crm_team_dataframe.to_csv(crm_team_file_path, index=False)

**Load crm.team**

In [ ]:
model_name = 'crm.team'
import_data(model_name)
import_data(model_name=model_name, ignore_fields=['member_ids'], file_csv=CRM_TEAM_MEMBERS_FILE_NAME, workers=1, context={'update_many2many': True})

**Extract and transform crm.lead**

In [ ]:
model_name = 'crm.lead'
export_data(model_name)

crm_lead_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
crm_lead_dataframe = pandas.read_csv(crm_lead_file_path)
replace_branches_ids(crm_lead_dataframe)
crm_lead_dataframe.rename(columns={'planned_revenue': 'expected_revenue'}, inplace=True)
crm_lead_dataframe['priority'] = crm_lead_dataframe['priority'].str.replace('Low', 'Medium')
crm_lead_dataframe['priority'] = crm_lead_dataframe['priority'].str.replace('Normal', 'Low')
crm_lead_dataframe['tag_ids/id'] = crm_lead_dataframe['tag_ids/id'].str.replace('False', '')
crm_lead_dataframe.sort_values('id', inplace=True)

crm_lead_dataframe.to_csv(crm_lead_file_path, index=False)

**Load crm.lead**

In [ ]:
import_data('crm.lead')

**Extract project.tags**

In [ ]:
export_data('project.tags')

**Load project.tags**

In [ ]:
import_data('project.tags')

**Extract and transform project.project**

In [ ]:
model_name = 'project.project'
export_data(model_name)

projects_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
projects_dataframe = pandas.read_csv(projects_file_path)
replace_branches_ids(projects_dataframe)
projects_dataframe['privacy_visibility'] = projects_dataframe['privacy_visibility'].str.replace('Visible by all employees', 'All employees')
projects_dataframe['privacy_visibility'] = projects_dataframe['privacy_visibility'].str.replace('On invitation only', 'Invited employees')
projects_dataframe.to_csv(projects_file_path, index=False)

**Load project.project**

In [ ]:
model_name = 'project.project'
import_data(model_name)

**Extract project.task.type**

In [ ]:
model_name = 'project.task.type'
export_data(model_name)

project_task_type_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
project_task_types_dataframe = pandas.read_csv(project_task_type_file_path)
project_task_types_dataframe['project_ids/id'] = project_task_types_dataframe['project_ids/id'].str.replace('False', '')
project_task_types_dataframe.to_csv(project_task_type_file_path, index=False)

**Load project.task.type**

In [ ]:
import_data('project.task.type')

**Extract and transform project.task**

In [ ]:
model_name = 'project.task'
export_data(model_name)

project_tasks_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
project_tasks_dataframe = pandas.read_csv(project_tasks_file_path)
project_tasks_dataframe['user_id/id'] = project_tasks_dataframe['user_id/id'].str.replace('False', '')
project_tasks_dataframe['tag_ids/id'] = project_tasks_dataframe['tag_ids/id'].astype(str).str.replace('False', '')
project_tasks_dataframe['tag_ids/id'] = project_tasks_dataframe['tag_ids/id'].str.replace('False', '')
project_tasks_dataframe['priority'] = project_tasks_dataframe['priority'].str.replace('Normal', 'Important')
project_tasks_dataframe['priority'] = project_tasks_dataframe['priority'].str.replace('Low', 'Normal')
project_tasks_dataframe.rename(columns={'user_id/id': 'user_ids/id'}, inplace=True)
project_tasks_dataframe.rename(columns={'email_from': 'email_cc'}, inplace=True)
project_tasks_dataframe.to_csv(project_tasks_file_path, index=False)

**Load project.task**

In [ ]:
model_name = 'project.task'
import_data(model_name)
import_ignored_fields(model_name, workers=1, group_by='parent_id/id')

**Extract mail.mass_mailing.contact**

In [ ]:
export_data('mail.mass_mailing.contact')

**Load mail.mass_mailing.contact**

In [ ]:
model_name = 'mail.mass_mailing.contact'

import_data(model_name)

**Extract and transform mail.mass_mailing.list**`

In [ ]:
export_data('mail.mass_mailing.list')

**Load mail.mass_mailing.list**

In [ ]:
model_name = 'mail.mass_mailing.list'

import_data(model_name)

**Extract  Mailing List Recipients**

In [ ]:
export_data('mail.mass_mailing.contact',
            fields=['id', 'opt_out', 'unsubscription_date', 'list_ids/id'],
            output_file='mailing.contact.subscription.csv')

recipients_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}mailing.contact.subscription.csv')
recipients_dataframe.rename(columns={
    'id': 'contact_id/id',
    'list_ids/id': 'list_id/id'
    }
  , inplace=True)

#Delete rows with empty list_id/id
for row in recipients_dataframe.iterrows():
    if row[1]['list_id/id'] == 'False':
        recipients_dataframe.drop(row[0], inplace=True)

#Add id column e.g: data_migration.mailing.contact.subscription_1
recipients_dataframe.insert(0, 'id', [f'data_migration.mailing.contact.subscription_{row_index}' for row_index in range(1, 1 + len(recipients_dataframe))])

recipients_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}mailing.contact.subscription.csv', index=False)



**Load mailing.contact.subscription**

In [ ]:
import_data(model_name='mailing.contact.subscription')

**Extract utm.source**

In [ ]:
export_data('utm.source')

**Load utm.source**

In [2]:
import_data('utm.source')

open generated_csv_files/utm.source.csv


open generated_csv_files/utm.source.csv


Skipping until line 0 excluded
time for batch [0] - 1000 of 546 : 6.599755048751831
546 utm.source imported, total time 6.600895643234253 second(s)


**Extract mail.mass_mailing**

In [ ]:
export_data('mail.mass_mailing')

mailings_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}mail.mass_mailing.csv')

mailings_dataframe['mailing_model_id/id'] = mailings_dataframe['mailing_model_id/id'].str.replace('mass_mailing.model_mail_mass_mailing_list', 'mass_mailing.model_mailing_list')
mailings_dataframe['mailing_model_id/id'] = mailings_dataframe['mailing_model_id/id'].str.replace('custom_authorize_net', 'base')
mailings_dataframe['contact_list_ids/id'] = mailings_dataframe['contact_list_ids/id'].str.replace('False', '')

mailings_dataframe['subject'] = mailings_dataframe['name']

mailings_dataframe['body_arch'] = mailings_dataframe['body_html']

mailings_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}mail.mass_mailing.csv', index=False)

**Load mail.mass_mailing**

In [ ]:
import_data('mail.mass_mailing')

**Extract mass.mailing.template**

In [ ]:
export_data('mass.mailing.template')

templates_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}mass.mailing.template.csv')

templates_dataframe['name'] = '[Broken, create this template manually] ' + templates_dataframe['name']
templates_dataframe['body_arch'] = templates_dataframe['body_html']

templates_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}mass.mailing.template.csv', index=False)


**Load mailing.template**

In [ ]:
import_data('mass.mailing.template')

**Extract and transform ir.model.data**
Requires all the previous steps to work properly given that it depends on the new external ids created on the new Odoo instance

In [ ]:
models_to_export_external_ids = ['crm.lead', 'crm.team', 'res.partner', 'project.project', 'project.task']
#models_to_export_external_ids = ['crm.lead', 'crm.team', 'res.partner', 'project.project', 'project.task', 'product.template', 'purchase.order', 'sale.order', 'sale.order.line']
#Export external_ids
fields_to_export = ['complete_name', 'res_id', 'model']
export_data(model_name='ir.model.data',
            fields=fields_to_export,
            output_file='ir.model.data.old.csv',
            domain=[['model', 'in', models_to_export_external_ids]]
            )
#Export v15 external_ids
export_data(config='import_connection.conf',
            model_name='ir.model.data',
            fields=fields_to_export,
            output_file='ir.model.data.new.csv',
            domain=[['model', 'in', models_to_export_external_ids]]
            )
external_ids_old_file_path = f'{GENERATED_CSV_FILES_PATH}ir.model.data.old.csv'
external_ids_new_file_path = f'{GENERATED_CSV_FILES_PATH}ir.model.data.new.csv'
merged_data_frame_file_path = f'{GENERATED_CSV_FILES_PATH}ir.model.data.merged.csv'
ir_model_data_old_dataframe = pandas.read_csv(external_ids_old_file_path)
ir_model_data_new_dataframe = pandas.read_csv(external_ids_new_file_path)
ir_model_data_old_dataframe['complete_name'] = ir_model_data_old_dataframe['complete_name'].str.replace('base.partner_root', 'base.partner_admin')
ir_model_data_old_dataframe['complete_name'] = ir_model_data_old_dataframe['complete_name'].str.replace('base.default_user_res_partner', 'base.template_portal_user_id_res_partner')
ir_model_data_old_dataframe.to_csv(external_ids_old_file_path, index=False)
# Merge old and new external ids to extract the new database id
ir_model_data_old_dataframe.rename(columns={'res_id': 'old_res_id'}, inplace=True)
ir_model_data_merged_dataframe = ir_model_data_new_dataframe.merge(ir_model_data_old_dataframe, on=['complete_name', 'model'], how='inner')
ir_model_data_merged_dataframe.rename(columns={'res_id': 'new_res_id'}, inplace=True)
ir_model_data_merged_dataframe['old_res_id'] = ir_model_data_merged_dataframe['old_res_id'].astype('int')
ir_model_data_merged_dataframe.to_csv(merged_data_frame_file_path, index=False)

**Extract and transform mail.message**
Requires all the previous steps to work properly given that it depends on the new external ids created on the new Odoo instance

In [ ]:
model_name = 'mail.message'
export_data(model_name)

mail_message_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
mail_message_dataframe = pandas.read_csv(mail_message_file_path)

mail_message_dataframe['subtype_id/id'] = mail_message_dataframe['subtype_id/id'].str.replace('False', '')
mail_message_dataframe['partner_ids/id'] = mail_message_dataframe['partner_ids/id'].astype('str')
mail_message_dataframe['partner_ids/id'] = mail_message_dataframe['partner_ids/id'].str.replace('False', '')

ir_model_data_merged_file_path = f'{GENERATED_CSV_FILES_PATH}ir.model.data.merged.csv'
ir_model_data_merged_dataframe = pandas.read_csv(ir_model_data_merged_file_path)

# Merge the mail.message dataframe with ir_model_data_merged_dataframe to get the new database ids
mail_message_dataframe = mail_message_dataframe.merge(ir_model_data_merged_dataframe, left_on=['model', 'res_id'], right_on=['model', 'old_res_id'], how='inner')
mail_message_dataframe.rename(columns={'res_id': 'old_res_id'}, inplace=True)
mail_message_dataframe.rename(columns={'new_res_id': 'res_id'}, inplace=True)
mail_message_dataframe['res_id'] = mail_message_dataframe['res_id'].astype('int')

mail_message_dataframe.sort_values('date', ascending=True, inplace=True)
mail_message_dataframe.to_csv(mail_message_file_path, index=False)

**Load mail.message**

In [ ]:
model_name = 'mail.message'
ignored_fields = ['old_res_id', 'old_res_id2', 'complete_name', 'parent_id/id']
import_data(model_name=model_name, ignore_fields=ignored_fields, group_by='complete_name')

**Extract and transform mail.tracking.value**

In [ ]:
model_name = 'mail.tracking.value'
mail_tracking_value_all_file_name = f'{model_name}.all.csv'
mail_tracking_value_all_file_path = f'{GENERATED_CSV_FILES_PATH}{mail_tracking_value_all_file_name}'
mail_tracking_value_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
export_data(model_name=model_name, output_file=mail_tracking_value_all_file_name)
mail_tracking_value_dataframe = pandas.read_csv(mail_tracking_value_all_file_path, low_memory=False)

mail_message_file_path = f'{GENERATED_CSV_FILES_PATH}mail.message.csv'
mail_message_dataframe = pandas.read_csv(mail_message_file_path, low_memory=False)
mail_message_dataframe.rename(columns={'id': 'mail_message_id/id'}, inplace=True)

#Merge tracking values with mail.message to filter the tracking values to import and adding the model column
mail_message_dataframe = mail_message_dataframe[['model', 'mail_message_id/id']]
mail_tracking_value_dataframe = mail_tracking_value_dataframe.merge(mail_message_dataframe, on='mail_message_id/id', how='inner')

#Export v15 ir_model_fields
fields_model_name = 'ir.model.fields'
models_with_mail_messages = list(mail_message_dataframe['model'].unique())
models_to_export_external_ids = [m for m in models_with_mail_messages if m in models_migration_config.keys()]
export_data(config='import_connection.conf',
            model_name=fields_model_name,
            fields=['id', 'name', 'model'],
            output_file=f'{fields_model_name}.csv',
            domain=[['model', 'in', models_to_export_external_ids]]
            )
ir_model_fields_file_path = f'{GENERATED_CSV_FILES_PATH}{fields_model_name}.csv'
fields_dataframe = pandas.read_csv(ir_model_fields_file_path)
fields_dataframe.rename(columns={'id': 'field/id'}, inplace=True)
fields_dataframe.rename(columns={'name': 'field'}, inplace=True)

#Merge tracking values with fields to extract the fields' external ids
mail_tracking_value_dataframe['field'] = mail_tracking_value_dataframe['field'].str.replace('planned_revenue', 'expected_revenue')
mail_tracking_value_dataframe['field'] = mail_tracking_value_dataframe['field'].str.replace('categ_id', 'category_id')
mail_tracking_value_dataframe['field'] = mail_tracking_value_dataframe['field'].str.replace('salesperson_ids', 'user_id')
mail_tracking_value_dataframe = mail_tracking_value_dataframe.merge(fields_dataframe, on=['field', 'model'], how='inner')
# Drop columns used just to merge
mail_tracking_value_dataframe.drop(columns={'field', 'model'}, inplace=True)

mail_tracking_value_dataframe.to_csv(mail_tracking_value_file_path, index=False)

**Load mail.tracking.value**

In [ ]:
import_data('mail.tracking.value')

**Extract ir.attachment**

In [ ]:
model_name = 'ir.attachment'
export_data(model_name)

ir_attachment_file_path = f'{GENERATED_CSV_FILES_PATH}{model_name}.csv'
ir_attachment_dataframe = pandas.read_csv(ir_attachment_file_path)

ir_model_data_merged_file_path = f'{GENERATED_CSV_FILES_PATH}ir.model.data.merged.csv'
ir_model_data_merged_dataframe = pandas.read_csv(ir_model_data_merged_file_path)
ir_model_data_merged_dataframe.rename(columns={'model': 'res_model'}, inplace=True)

# Merge the ir.attachment dataframe with ir_model_data_merged_dataframe to get the new database ids
ir_attachment_dataframe.rename(columns={'res_id': 'old_res_id'}, inplace=True)
ir_attachment_dataframe = ir_attachment_dataframe.merge(ir_model_data_merged_dataframe, on=['res_model', 'old_res_id'], how='inner')
ir_attachment_dataframe.drop(columns={'old_res_id', 'complete_name'}, inplace=True)
ir_attachment_dataframe.rename(columns={'new_res_id': 'res_id'}, inplace=True)

ir_attachment_dataframe.sort_values('id', ascending=True, inplace=True)
ir_attachment_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}{model_name}2.csv', index=False)

**Extract ir.attachment** Part 2
We separate this process into 2 parts to keep the output shorter, in part 1 all attachments references are loaded but with empty files, in this second part the actual files data is loaded

In [ ]:
#Separate into batches per model to keep files smaller and avoid memory errors
attachments_model_name = 'ir.attachment'
fields_to_export = ['id', 'name', 'datas']
# models_with_attachments = ['project.task']
models_with_attachments = ['res.partner']
max_file_size_for_batches = 50000000 # 50 MB
batch_size = 20 # Some files are very large so it fails due to 413 (Payload Too Large), so we keep the batch size small
#Batch sizes 15 10 5 3
# export_data(
#     attachments_model_name,
#     fields=fields_to_export,
#     domain=[['res_model', 'in', models_with_attachments], ['file_size', '>', max_file_size_for_batches]],
#     output_file=f'{attachments_model_name}.big_files.csv',
#     batch_size=1
# )
for model_name in models_with_attachments:
    export_data(
        attachments_model_name,
        fields=fields_to_export,
        domain=[['res_model', '=', model_name], ['file_size', '<', max_file_size_for_batches]],
        output_file=f'{attachments_model_name}.{model_name}.csv',
        batch_size=3
    )
        # export_data(
        #     attachments_model_name,
        #     fields=fields_to_export,
        #     domain=[['res_model', '=', model_name], ['file_size', '<', max_file_size_for_batches], ['file_size', '>', max_file_size_for_batches/4]],
        #     output_file=f'{attachments_model_name}.{model_name}2.csv',
        #     batch_size=15
        # )
    # else:
    #     export_data(
    #         attachments_model_name,
    #         fields=fields_to_export,
    #         domain=[['res_model', '=', model_name], ['file_size', '<', max_file_size_for_batches]],
    #         output_file=f'{attachments_model_name}.{model_name}.csv',
    #         batch_size=15
    #     )

**Load ir.attachment**

In [ ]:
attachments_model_name = 'ir.attachment'
batch_size = 15
models_with_attachments = ['project.task']
#'project.task' 'crm.lead'
#import_data(attachments_model_name, file_csv=f'{attachments_model_name}2.csv')
#import_data(attachments_model_name, file_csv=f'{attachments_model_name}.big_files.csv', batch_size=1)
for model_name in models_with_attachments:
    if model_name == 'project.task':
        # TODO: Run 4
        #import_data(attachments_model_name, file_csv=f'ir.attachment.project.task1.csv.fail', batch_size=2)
        
        import_data(attachments_model_name, file_csv=f'{attachments_model_name}.{model_name}3.csv', batch_size=2)
